In [ ]:
import os
import zipfile
import csv

from collections import defaultdict, Counter

In [ ]:
path_to_data = os.path.join("data", "input")
path_out = os.path.join("data", "processed.csv")

In [ ]:
# to_exclude = ["18_0{}.zip".format(x) for x in range(1, 6)]
to_exclude = []

In [ ]:
produkte = Counter()
betreiber_names_per_id = defaultdict(set)
haltestellen_names_per_uic = defaultdict(set)
with open(path_out, "wb") as out:
    for filename in os.listdir(path_to_data):
        if filename not in to_exclude:
            csv_writer = csv.writer(out, delimiter=";")
            header = ["BETRIEBSTAG", "FAHRT_BEZEICHNER", "BETREIBER_ID", "VERKEHRSMITTEL_TEXT", "BPUIC", "ANKUNFTSZEIT", "AN_PROGNOSE", "ABFAHRTSZEIT", "AB_PROGNOSE"]
            csv_writer.writerow(header)
            if filename.endswith(".zip"):
                print filename
                with zipfile.ZipFile(os.path.join(path_to_data, filename)) as f:
                    for name in f.namelist():
                        with f.open(name) as ff:
                            reader = csv.DictReader(ff, delimiter=";")
                            nb_written = 0
                            for line in reader:
                                produkt = line['PRODUKT_ID']
                                produkte[produkt] += 1
                                an_status = line['AN_PROGNOSE_STATUS']
                                ab_status = line['AB_PROGNOSE_STATUS']
                                if produkt == 'Zug' and ((an_status == 'REAL') or (ab_status == 'REAL')):
                                    bpuic = line["BPUIC"]
                                    haltestellen_names_per_uic[bpuic].add(line["HALTESTELLEN_NAME"])
                                    betreiber_id = line["BETREIBER_ID"]
                                    betreiber_names_per_id[betreiber_id].add(line["BETREIBER_NAME"])
                                    an = line['AN_PROGNOSE']
                                    an_real = an if an_status == "REAL" else ""
                                    ab = line['AB_PROGNOSE']
                                    ab_real = ab if ab_status == "REAL" else ""
                                    csv_writer.writerow([
                                        line["BETRIEBSTAG"], 
                                        line["FAHRT_BEZEICHNER"], 
                                        line["BETREIBER_ID"], 
                                        line["VERKEHRSMITTEL_TEXT"],
                                        bpuic,
                                        line["ANKUNFTSZEIT"],
                                        an_real,
                                        line["ABFAHRTSZEIT"],
                                        ab_real
                                    ])
                                    nb_written += 1
                        out.flush()
                        print "   {} processed. # lines written: {}".format(name, nb_written)

In [ ]:
def write_to_csv(list_per_id, header, path_out):
    with open(path_out, "wb") as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(header)
        for _id, entry_list in list_per_id.items():
            for i, entry in enumerate(entry_list):
                writer.writerow([_id, i, entry])

In [ ]:
path_out_betreiber = os.path.join("data", "betreiber.csv")
header_betreiber = ["BETREIBER_ID", "INDEX", "BETREIBER_NAME"]
write_to_csv(betreiber_names_per_id, header_betreiber, path_out_betreiber)

In [ ]:
path_out_haltestellen = os.path.join("data", "haltestellen.csv")
header_haltestellenr = ["BPUIC", "INDEX", "HALTESTELLEN_NAME"]
write_to_csv(haltestellen_names_per_uic, header_haltestellenr, path_out_haltestellen)